# Scratchwork

In [1]:
from finite_algebras import *
from cayley_table import *
from permutations import *

In [2]:
import os
aa_path = os.path.join(os.getenv("PYPROJ"), "abstract_algebra")
alg_dir = os.path.join(aa_path, "Algebras")

In [3]:
ex = Examples(alg_dir)

                           Example Algebras
----------------------------------------------------------------------
  13 example algebras are available.
  Use "get_example(INDEX)" to retrieve a specific example,
  where INDEX is the first number on each line below:
----------------------------------------------------------------------
0: A4 -- Alternating group on 4 letters (AKA Tetrahedral group)
1: D3 -- https://en.wikipedia.org/wiki/Dihedral_group_of_order_6
2: D4 -- Dihedral group on four vertices
3: Pinter29 -- Non-abelian group, p.29, 'A Book of Abstract Algebra' by Charles C. Pinter
4: RPS -- Rock, Paper, Scissors Magma
5: S3 -- Symmetric group on 3 letters
6: S3X -- Another version of the symmetric group on 3 letters
7: V4 -- Klein-4 group
8: Z4 -- Cyclic group of order 4
9: F4 -- Field with 4 elements (from Wikipedia)
10: mag_id -- Magma with Identity
11: Example 1.4.1 -- See: Groupoids and Smarandache Groupoids by W. B. Vasantha Kandasamy
12: Ex6 -- Example 6: http://www-group

In [4]:
sg = ex.get_example(11)
sg.about()


Semigroup: Example 1.4.1
Instance ID: 140659125926928
Description: See: Groupoids and Smarandache Groupoids by W. B. Vasantha Kandasamy
Elements: ['a', 'b', 'c', 'd', 'e', 'f']
Identity: None
Associative? Yes
Commutative? No
Has Inverses? No
Cayley Table (showing indices):
[[0, 3, 0, 3, 0, 3],
 [1, 4, 1, 4, 1, 4],
 [2, 5, 2, 5, 2, 5],
 [3, 0, 3, 0, 3, 0],
 [4, 1, 4, 1, 4, 1],
 [5, 2, 5, 2, 5, 2]]


In [5]:
def about_subalgebra_partitions(alg, iso_partitions):
    print("\n" + "="*70)
    print(f"Subalgebras of {alg}")
    print(f"   Partitioned into isomorphically equivalent subalgebras:")
    print("="*70)
    alg_has_inverses = alg.has_inverses()
    for partition in iso_partitions:
        for sub in partition:
            print(f"\n{sub}")
            print(f"      Order: {sub.order}")
            print(f"   Elements: {sub.elements}")
            if sub.has_identity():
                print(f"   Identity: {sub.identity}")
                print(f"Commutative: {yes_or_no(sub.is_commutative())}")
                if alg_has_inverses:
                    print(f"     Normal: {yes_or_no(alg.is_normal(sub))}")
        print("\n" + "-"*60)

In [6]:
def about_partition(alg, part):
    
    size = len(part)
    sub0 = part[0]
    classname = f"Sub-{sub0.__class__.__name__}"
    order = sub0.order
    has_identity = sub0.has_identity()
    
    #identities = False
    #if sub0.has_identity():
    #    identities = {sub.identity for sub in part}
    #    single_id = False
    #    if len(identities) == 1:
    #        single_id = True
    #if identities:
    #    print(f"Single ID? {single_id}")
    
    comm = ""
    if sub0.is_commutative():
        comm = "Commutative "
        
    norm = ""
    if alg.has_inverses() and alg.is_normal(sub0):
        norm = "Normal "
        
    if size > 1:
        print(f"\n{size} Isomorphic {comm}{norm}{classname}s of Order {order}:")
        for sub in part:
            print(f"      {sub.name}: {sub.elements}")
    elif size == 1:
        print(f"\nA {comm}{norm}{classname} of Order {order}:")
        print(f"      {sub0.name}: {sub0.elements}")
    else:
        raise ValueError("A partition must have at least one member.")

def about_partitions(alg, partitions):
    for partition in partitions:
        about_partition(alg, partition)

In [7]:
alg = sg
alg_subs = alg.proper_subalgebras(False)
partitions = partition_into_isomorphic_lists(alg_subs)
#about_subalgebra_partitions(alg, partitions)
about_partitions(sg, partitions)


3 Isomorphic Commutative Sub-Groups of Order 2:
      Example 1.4.1_subalgebra_0: ['a', 'd']
      Example 1.4.1_subalgebra_6: ['b', 'e']
      Example 1.4.1_subalgebra_8: ['c', 'f']

3 Isomorphic Sub-Semigroups of Order 4:
      Example 1.4.1_subalgebra_1: ['a', 'c', 'd', 'f']
      Example 1.4.1_subalgebra_3: ['b', 'c', 'e', 'f']
      Example 1.4.1_subalgebra_5: ['a', 'b', 'd', 'e']

3 Isomorphic Sub-Semigroups of Order 2:
      Example 1.4.1_subalgebra_2: ['a', 'e']
      Example 1.4.1_subalgebra_4: ['c', 'e']
      Example 1.4.1_subalgebra_7: ['a', 'c']

A Sub-Semigroup of Order 3:
      Example 1.4.1_subalgebra_9: ['a', 'c', 'e']


In [8]:
a4 = ex.get_example(0)
a4

Group(
'A4',
'Alternating group on 4 letters (AKA Tetrahedral group)',
['()', '(1 3 2)', '(1 2 3)', '(0 1)(2 3)', '(0 1 2)', '(0 1 3)', '(0 3 1)', '(0 3 2)', '(0 3)(1 2)', '(0 2 3)', '(0 2 1)', '(0 2)(1 3)'],
[[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11], [1, 2, 0, 6, 7, 8, 10, 11, 9, 5, 3, 4], [2, 0, 1, 10, 11, 9, 3, 4, 5, 8, 6, 7], [3, 4, 5, 0, 1, 2, 9, 10, 11, 6, 7, 8], [4, 5, 3, 9, 10, 11, 7, 8, 6, 2, 0, 1], [5, 3, 4, 7, 8, 6, 0, 1, 2, 11, 9, 10], [6, 7, 8, 1, 2, 0, 5, 3, 4, 10, 11, 9], [7, 8, 6, 5, 3, 4, 11, 9, 10, 0, 1, 2], [8, 6, 7, 11, 9, 10, 1, 2, 0, 4, 5, 3], [9, 10, 11, 4, 5, 3, 2, 0, 1, 7, 8, 6], [10, 11, 9, 2, 0, 1, 8, 6, 7, 3, 4, 5], [11, 9, 10, 8, 6, 7, 4, 5, 3, 1, 2, 0]]
)

In [9]:
a4.about()


Group: A4
Instance ID: 140659381787152
Description: Alternating group on 4 letters (AKA Tetrahedral group)
Identity: ()
Associative? Yes
Commutative? No
Elements:
   Index   Name   Inverse  Order
      0      ()      ()       1
      1 (1 3 2) (1 2 3)       3
      2 (1 2 3) (1 3 2)       3
      3 (0 1)(2 3) (0 1)(2 3)       2
      4 (0 1 2) (0 2 1)       3
      5 (0 1 3) (0 3 1)       3
      6 (0 3 1) (0 1 3)       3
      7 (0 3 2) (0 2 3)       3
      8 (0 3)(1 2) (0 3)(1 2)       2
      9 (0 2 3) (0 3 2)       3
     10 (0 2 1) (0 1 2)       3
     11 (0 2)(1 3) (0 2)(1 3)       2
Cayley Table (showing indices):
[[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11],
 [1, 2, 0, 6, 7, 8, 10, 11, 9, 5, 3, 4],
 [2, 0, 1, 10, 11, 9, 3, 4, 5, 8, 6, 7],
 [3, 4, 5, 0, 1, 2, 9, 10, 11, 6, 7, 8],
 [4, 5, 3, 9, 10, 11, 7, 8, 6, 2, 0, 1],
 [5, 3, 4, 7, 8, 6, 0, 1, 2, 11, 9, 10],
 [6, 7, 8, 1, 2, 0, 5, 3, 4, 10, 11, 9],
 [7, 8, 6, 5, 3, 4, 11, 9, 10, 0, 1, 2],
 [8, 6, 7, 11, 9, 10, 1, 2, 0, 4, 5, 3],


In [10]:
alg = a4
alg_subs = alg.proper_subalgebras(False)
partitions = partition_into_isomorphic_lists(alg_subs)
about_partitions(alg, partitions)


4 Isomorphic Commutative Sub-Groups of Order 3:
      A4_subalgebra_0: ['()', '(0 1 3)', '(0 3 1)']
      A4_subalgebra_1: ['()', '(0 1 2)', '(0 2 1)']
      A4_subalgebra_3: ['()', '(0 3 2)', '(0 2 3)']
      A4_subalgebra_6: ['()', '(1 3 2)', '(1 2 3)']

A Commutative Normal Sub-Group of Order 4:
      A4_subalgebra_2: ['()', '(0 1)(2 3)', '(0 3)(1 2)', '(0 2)(1 3)']

3 Isomorphic Commutative Sub-Groups of Order 2:
      A4_subalgebra_4: ['()', '(0 3)(1 2)']
      A4_subalgebra_5: ['()', '(0 1)(2 3)']
      A4_subalgebra_7: ['()', '(0 2)(1 3)']


In [11]:
f4 = ex.get_example(9)
f4.about()


Field: F4
Instance ID: 140659125925712
Description: Field with 4 elements (from Wikipedia)
Identity: 0
Associative? Yes
Commutative? Yes
Elements:
   Index   Name   Inverse  Order
      0       0       0       1
      1       1       1       2
      2       a       a       2
      3     1+a     1+a       2
Cayley Table (showing indices):
[[0, 1, 2, 3], [1, 0, 3, 2], [2, 3, 0, 1], [3, 2, 1, 0]]
Mult. Identity: 1
Mult. Commutative? Yes
Multiplicative Cayley Table (showing indices):
[[0, 0, 0, 0], [0, 1, 2, 3], [0, 2, 3, 1], [0, 3, 1, 2]]


In [12]:
alg = f4
alg_subs = alg.proper_subalgebras(False)
partitions = partition_into_isomorphic_lists(alg_subs)
about_partitions(alg, partitions)


3 Isomorphic Commutative Normal Sub-Groups of Order 2:
      F4_subalgebra_0: ['0', '1+a']
      F4_subalgebra_1: ['0', 'a']
      F4_subalgebra_2: ['0', '1']


In [13]:
ex6 = ex.get_example(12)
ex6

Ring(
'Ex6',
'Example 6: http://www-groups.mcs.st-andrews.ac.uk/~john/MT4517/Lectures/L3.html',
['0', 'a', 'b', 'c'],
[[0, 1, 2, 3], [1, 0, 3, 2], [2, 3, 0, 1], [3, 2, 1, 0]],
[[0, 0, 0, 0], [0, 0, 1, 1], [0, 0, 2, 2], [0, 0, 3, 3]]
)

In [14]:
psr = generate_powerset_group(4)
psr.order

16

In [15]:
psr.about()


Group: PS4
Instance ID: 140660179487504
Description: Autogenerated Group on the powerset of 4 elements, with symmetric difference operator
Identity: {}
Associative? Yes
Commutative? Yes
Elements:
   Index   Name   Inverse  Order
      0      {}      {}       1
      1     {0}     {0}       2
      2     {1}     {1}       2
      3     {2}     {2}       2
      4     {3}     {3}       2
      5  {0, 1}  {0, 1}       2
      6  {0, 2}  {0, 2}       2
      7  {0, 3}  {0, 3}       2
      8  {1, 2}  {1, 2}       2
      9  {1, 3}  {1, 3}       2
     10  {2, 3}  {2, 3}       2
     11 {0, 1, 2} {0, 1, 2}       2
     12 {0, 1, 3} {0, 1, 3}       2
     13 {0, 2, 3} {0, 2, 3}       2
     14 {1, 2, 3} {1, 2, 3}       2
     15 {0, 1, 2, 3} {0, 1, 2, 3}       2
Group order is 16 > 12, so no further info calculated/printed.


In [16]:
alg = psr
alg_subs = alg.proper_subalgebras(True)
partitions = partition_into_isomorphic_lists(alg_subs)
about_partitions(alg, partitions)


35 Isomorphic Commutative Normal Sub-Groups of Order 4:
      PS4_subalgebra_0: ['{}', '{0}', '{3}', '{0, 3}']
      PS4_subalgebra_1: ['{}', '{0, 1}', '{2, 3}', '{0, 1, 2, 3}']
      PS4_subalgebra_2: ['{}', '{1}', '{2, 3}', '{1, 2, 3}']
      PS4_subalgebra_3: ['{}', '{0, 1}', '{0, 2, 3}', '{1, 2, 3}']
      PS4_subalgebra_4: ['{}', '{0, 3}', '{0, 1, 2}', '{1, 2, 3}']
      PS4_subalgebra_10: ['{}', '{0, 1}', '{0, 3}', '{1, 3}']
      PS4_subalgebra_11: ['{}', '{1, 3}', '{0, 1, 2}', '{0, 2, 3}']
      PS4_subalgebra_12: ['{}', '{1, 2}', '{1, 3}', '{2, 3}']
      PS4_subalgebra_13: ['{}', '{3}', '{0, 1}', '{0, 1, 3}']
      PS4_subalgebra_15: ['{}', '{0, 2}', '{0, 1, 3}', '{1, 2, 3}']
      PS4_subalgebra_18: ['{}', '{3}', '{0, 2}', '{0, 2, 3}']
      PS4_subalgebra_22: ['{}', '{0}', '{1, 3}', '{0, 1, 3}']
      PS4_subalgebra_23: ['{}', '{2}', '{3}', '{2, 3}']
      PS4_subalgebra_25: ['{}', '{0}', '{2, 3}', '{0, 2, 3}']
      PS4_subalgebra_26: ['{}', '{0}', '{1}', '{0, 1}']
      

In [17]:
alg = psr
alg_subs = alg.proper_subalgebras(False)
partitions = partition_into_isomorphic_lists(alg_subs)
about_partitions(alg, partitions)


35 Isomorphic Commutative Normal Sub-Groups of Order 4:
      PS4_subalgebra_0: ['{}', '{0}', '{3}', '{0, 3}']
      PS4_subalgebra_1: ['{}', '{0, 1}', '{2, 3}', '{0, 1, 2, 3}']
      PS4_subalgebra_2: ['{}', '{1}', '{2, 3}', '{1, 2, 3}']
      PS4_subalgebra_3: ['{}', '{0, 1}', '{0, 2, 3}', '{1, 2, 3}']
      PS4_subalgebra_4: ['{}', '{0, 3}', '{0, 1, 2}', '{1, 2, 3}']
      PS4_subalgebra_10: ['{}', '{0, 1}', '{0, 3}', '{1, 3}']
      PS4_subalgebra_11: ['{}', '{1, 3}', '{0, 1, 2}', '{0, 2, 3}']
      PS4_subalgebra_12: ['{}', '{1, 2}', '{1, 3}', '{2, 3}']
      PS4_subalgebra_13: ['{}', '{3}', '{0, 1}', '{0, 1, 3}']
      PS4_subalgebra_15: ['{}', '{0, 2}', '{0, 1, 3}', '{1, 2, 3}']
      PS4_subalgebra_18: ['{}', '{3}', '{0, 2}', '{0, 2, 3}']
      PS4_subalgebra_22: ['{}', '{0}', '{1, 3}', '{0, 1, 3}']
      PS4_subalgebra_23: ['{}', '{2}', '{3}', '{2, 3}']
      PS4_subalgebra_25: ['{}', '{0}', '{2, 3}', '{0, 2, 3}']
      PS4_subalgebra_26: ['{}', '{0}', '{1}', '{0, 1}']
      

In [18]:
# %time foo1 = psr.closed_subsets_of_elements(False)

In [19]:
# {len(x) for x in foo1}

In [20]:
# %time foo2 = psr.closed_subsets_of_elements(True)

In [21]:
# {len(x) for x in foo2}

In [22]:
# %time psr_subs = psr.unique_proper_subgroups()

In [23]:
# [psr_sub.order for psr_sub in psr_subs]